In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 18 17:55:08 2020

@author: Pilin
"""

#%%
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
import itertools
import mne
from mne.decoding import CSP
from mne import Epochs
from mne.decoding import SPoC
mne.set_log_level(verbose='warning') #to avoid info at terminal
import pickle
import sys
import IO
import os
import multiprocessing
from threading import Thread
from queue import Queue
#import tensorflow
#import tensorflow as tf
#import keras
#from keras.layers import BatchNormalization
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
#from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
#from keras.optimizers import Adam

import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
#from tensorflow.keras import backend as K
from tensorflow.compat.v1.keras import backend as K
from keras import backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

#from tensorflow.python.keras import backend as K
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
#from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint  did NOT work
#from keras.models import load_model
#from tensorflow.keras.layers import Dense, Activation, Permute, Dropout

from scipy import stats
from collections import OrderedDict
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from bayes_opt import BayesianOptimization
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
import gc
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBRegressor

VICTORIA = False
WRITE_OUT_CH_IND = False
USED_MODEL = 2 # 0 - Enet, 1 - XGB, 2 - NN
settings = {}
VERBOSE_ALL = 0

if VICTORIA is True:
    # insert at 1, 0 is the script path (or '' in REPL)
    sys.path.insert(1, '/home/victoria/icn/icn_m1')
    settings['BIDS_path'] = "//mnt/Datos/BML_CNCRS/Data_BIDS_new/"
    settings['out_path'] = "/mnt/Datos/BML_CNCRS/Data_processed/Derivatives/Raw_pipeline/"
    if USED_MODEL==0 :
           settings['out_path_process'] = "/mnt/Datos/BML_CNCRS/Spoc/ECoG_STN/LM_Out/"
    if USED_MODEL==1 :
           settings['out_path_process'] = "/mnt/Datos/BML_CNCRS/Spoc/ECoG_STN/XGB_Out/"
else:
    settings['BIDS_path'] = "C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\\MOVEMENT DATA\\"
    settings['out_path'] = "C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\\MOVEMENT DATA\\derivatives\\Int_old_grid\\"
    settings['out_path_process'] = "C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\MOVEMENT DATA\\ECoG_STN\\NN_Out\\"

settings['frequencyranges']=[[4, 8], [8, 12], [13, 20], [20, 35], [13, 35], [60, 80], [90, 200], [60, 200]]
settings['seglengths']=[1, 2, 2, 3, 3, 3, 10, 10, 10]
settings['num_patients']=['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014']
settings['BIDS_path']=settings['BIDS_path'].replace("\\", "/")
settings['out_path']=settings['out_path'].replace("\\", "/")

Using TensorFlow backend.


### test 1: does pool_function_la run through? 



In [2]:
def create_model_NN(learning_rate, num_dense_layers, num_input_nodes, num_dense_nodes, activation):
        """
        Create NN tensorflow with different numbers of hidden layers / hidden units
        """

        #start the model making process and create our first layer
        model = tensorflow.keras.Sequential()
        model.add(Dense(num_input_nodes, input_shape=(40,), activation=activation))

        #create a loop making a new dense layer for the amount passed to this model.
        #naming the layers helps avoid tensorflow error deep in the stack trace.
        for i in range(num_dense_layers):
            name = 'layer_dense_{0}'.format(i+1)
            model.add(BatchNormalization())
            model.add(Dropout(0.2))
            model.add(Dense(num_dense_nodes,
                     activation=activation,
                            name=name
                     ))
        #add our classification layer.
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        model.add(Dense(1,activation='linear'))

        #setup our optimizer and compile
        adam = Adam(lr=learning_rate)
        model.compile(optimizer=adam, loss='mean_squared_error',
                     metrics=['mse'])
        return model


def append_time_dim(arr, y_, time_stamps):
    """
    apply added time dimension for the data array and label given time_stamps (with downsample_rate=100) in 100ms / need to check with 1375Hz
    """
    time_arr = np.zeros([arr.shape[0]-time_stamps, int(time_stamps*arr.shape[1])])
    for time_idx, time_ in enumerate(np.arange(time_stamps, arr.shape[0])):
        for time_point in range(time_stamps):
            time_arr[time_idx, time_point*arr.shape[1]:(time_point+1)*arr.shape[1]] = arr[time_-time_point,:]
    return time_arr, y_[time_stamps:]

In [10]:
cv = KFold(n_splits=3, shuffle=True)

In [3]:
def pool_function_la(X, label, ch_idx, laterality_, signal_, subfolder, sess_idx, sub_idx):

    Ypre_te= []
    Ypre_tr= []
    score_tr= []
    Ypre_te= []
    score_te= []
    label_test=[]
    label_train=[]
    coords = []
    coef_ = []
    hyp_=[]
    for train_index, test_index in cv.split(X):
        Xtr, Xte=X[train_index,:], X[test_index,:]
        Ytr, Yte=label[train_index], label[test_index]
        label_test.append(Yte)
        label_train.append(Ytr)
        dat_tr,label_tr = append_time_dim(Xtr, Ytr, time_stamps=5)
        dat_te,label_te = append_time_dim(Xte, Yte, time_stamps=5)

        if USED_MODEL == 0: # Enet
            optimizer=optimize_enet(x=dat_tr,y=label_tr)
            model=ElasticNet(alpha=optimizer['x'][0],
                               l1_ratio=optimizer['x'][1],
                               max_iter=1000,
                               normalize=False)
        elif USED_MODEL == 1: # XGB
            optimizer=optimize_xgb(x=dat_tr, y=label_tr)
            model=XGBRegressor(max_depth=optimizer['x'][0],
                               learning_rate=optimizer['x'][1],
                               gamma=optimizer['x'][2], subsample= 0.8,
                               eta= 0.1,
                               disable_default_eval_metric= 1)

        elif USED_MODEL == 2:
            try:
                #optimizer = optimize_nn(dat_tr, label_tr, ch_idx, laterality_)
                optimizer = {}
                optimizer['x'] = {}
                optimizer['x'][0] = 0.01
                optimizer['x'][1] = 1
                optimizer['x'][2] = 2
                optimizer['x'][3] = 2
                optimizer['x'][4] = "tanh"
            except:
                print("INF / NAN ERROR")
                continue
            learning_rate=optimizer['x'][0]
            num_dense_layers=optimizer['x'][1]
            num_input_nodes=optimizer['x'][2]
            num_dense_nodes=optimizer['x'][3]
            activation=optimizer['x'][4]
            model = create_model_NN(learning_rate, num_dense_layers, num_input_nodes, num_dense_nodes, activation)
        else:
            break
            print("ARCHITECTURE IS NOT DEFINED")

        if USED_MODEL == 2:

            es = EarlyStopping(monitor='val_mse', mode='min', verbose=VERBOSE_ALL, patience=10)
            mc = ModelCheckpoint('best_model.h5', monitor='val_mse', mode='min', verbose=VERBOSE_ALL, save_best_only=True)
            X_train, X_val, y_train, y_val = train_test_split(dat_tr, label_tr, train_size=0.8,shuffle=True)
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, batch_size=100, verbose=VERBOSE_ALL, callbacks=[mc,es])
            r2_tr = metrics.r2_score(model.predict(X_train), y_train)
            if r2_tr < 0: r2_tr = 0
            r2_te = metrics.r2_score(model.predict(dat_te), label_te)
            print("channel: "+str(ch_idx)+" r2 test: "+str(r2_te))
            if r2_te < 0: r2_te = 0

        else:
            model.fit(dat_tr, label_tr)
            r2_tr=model.score(dat_tr, label_tr)
            if r2_tr < 0: r2_tr = 0
            r2_te=model.score(dat_te, label_te)
            if r2_te < 0: r2_te = 0

        score_tr.append(r2_tr)
        score_te.append(r2_te)
        #with tf.device(tf.DeviceSpec(device_type="CPU", device_index=0)):
        Ypre_te.append(model.predict(dat_te) if USED_MODEL != 2 else model.predict(dat_te)[:,0])
        Ypre_tr.append(model.predict(dat_tr) if USED_MODEL != 2 else model.predict(dat_tr)[:,0])
        if USED_MODEL == 0: coef_.append(model.coef_)
        hyp_.append(optimizer['x'])

    Score_tr=np.mean(score_tr)
    Score_te=np.mean(score_te)
    Label_te=label_test
    Label_tr=label_train
    Labelpre_te=Ypre_te
    Labelpre_tr=Ypre_tr
    COEF_=coef_
    Hyperarapms=hyp_

    predict_ = {
        "y_pred_test": Ypre_te,
        "y_test": Label_te,
        "y_pred_train": Ypre_tr,
        "y_train": Label_tr,
        "score_tr": Score_tr,
        "score_te": Score_te,
        #"coord_patient" : run_["coord_patient"],
        "coef" :coef_,
        "model_hyperparams": hyp_
    }

    out_path_file = os.path.join(settings['out_path_process']+ \
        settings['num_patients'][sub_idx]+'BestChpredictions_'+\
        signal_+'-ch-'+str(ch_idx)+'-lat-'+str(laterality_)+'-'+str(subfolder[sess_idx])+'.npy')
    np.save(out_path_file, predict_)


In [32]:
X = np.random.random([1000,8])
label = np.arange(1000)
ch_idx = 0
laterality_ = "CON"
signal_ = "ECOG"
subfolder = ["right"]
sess_idx = 0
sub_idx = 0

In [33]:
pool_function_la(X, label, ch_idx, laterality_, signal_, subfolder, sess_idx, sub_idx)

channel: 0 r2 test: -459747.93160811736
channel: 0 r2 test: -6.120615195640106
channel: 0 r2 test: -22.600996426363935


### Rewrite getting data

In [4]:
def get_int_runs(subject_id, subfolder):
    """

    :param patient_idx:
    :return: list with all run files for the given patient
    """
    os.listdir(settings['out_path'])

    if 'right' in str(subfolder):
        list_subject = [i for i in os.listdir(settings['out_path']) if i.startswith('sub_'+subject_id+'_sess_right') and i.endswith('.p')]
    else:
        list_subject = [i for i in os.listdir(settings['out_path']) if i.startswith('sub_'+subject_id+'_sess_left') and i.endswith('.p')]

    return list_subject

In [37]:
cv = KFold(n_splits=3, shuffle=False)
laterality=[("CON"), ("IPS")]
signal=["ECOG", "STN"]

In [5]:
def get_patient_data():
    
    for sub_idx in np.arange(0, len(settings['num_patients']), 1):
        list_param = [] # list for pool
        for signal_idx, signal_ in enumerate(signal):
            subject_path=settings['BIDS_path'] + 'sub-' + settings['num_patients'][sub_idx]
            subfolder=IO.get_subfolders(subject_path)

            for sess_idx in range(len(subfolder)):
                if os.path.exists(os.path.join(settings['out_path_process'],
                            settings['num_patients'][sub_idx]+'BestChpredictions_'+signal_+'-'+
                                str(subfolder[sess_idx])+'.npy')) is True:
                    continue
                X=[]
                Y_con=[]
                Y_ips=[]
                list_subject=get_int_runs(settings['num_patients'][sub_idx], subfolder[sess_idx])
                list_subject=sorted(list_subject)
                if signal_=="ECOG":
                    if sub_idx==4 and sess_idx==0: #for sake of comparison with spoc
                        list_subject.pop(0)
                    if sub_idx==4 and sess_idx==1:
                        list_subject.pop(2)

                print('RUNNIN SUBJECT_'+ settings['num_patients'][sub_idx]+ '_SESS_'+ str(subfolder[sess_idx]) + '_SIGNAL_' + signal_)
                for run_idx in range(len(list_subject)):
                    with open(settings['out_path']+ '/'+ list_subject[run_idx], 'rb') as handle:
                        run_ = pickle.load(handle)

                    #concatenate features
                    #get cortex data only
                    if signal_=="ECOG":
                        ind_cortex=run_['used_channels']['cortex']
                        rf=run_['rf_data_median']
                        x=rf[:,ind_cortex,:]
                        x=np.clip(x, -2,2) # this should have been implemented in the pipeline
                        y=run_['label_baseline_corrected']
                        con_true=run_['label_con_true']
                        y_con=np.squeeze(y[con_true==True])
                        y_ips=np.squeeze(y[con_true==False])
                        X.append(x)
                        Y_con.append(y_con)
                        Y_ips.append(y_ips)
                    else:
                        ind_subcortex=run_['used_channels']['subcortex']
                        if ind_subcortex is not None:

                            rf=run_['rf_data_median']
                            x=rf[:,ind_subcortex,:]
                            x=np.clip(x, -2,2)

                            y=run_['label_baseline_corrected']
                            con_true=run_['label_con_true']
                            y_con=np.squeeze(y[con_true==True])
                            y_ips=np.squeeze(y[con_true==False])

                            X.append(x)
                            Y_con.append(y_con)
                            Y_ips.append(y_ips)

                gc.collect() # free unreferenced memory
                X=np.concatenate(X, axis=0)
                Y_con=np.concatenate(Y_con, axis=0)
                Y_ips=np.concatenate(Y_ips, axis=0)

                for laterality_idx, laterality_ in enumerate(laterality):
                    for ch_idx in range(X.shape[1]):
                        if laterality_ == "CON":
                            label_here = Y_con
                        else:
                            label_here = Y_ips
                        yield X[:,ch_idx,:], label_here, ch_idx, laterality_, signal_, subfolder, sess_idx, sub_idx
                        #list_param.append((X[:,ch_idx,:], label_here, ch_idx, laterality_, signal_, subfolder, sess_idx, sub_idx))
        #pool = multiprocessing.Pool(len(list_param))
        #pool.starmap(pool_function_la, list_param)

In [52]:
l_all = []

In [53]:
patient_dat_generator = get_patient_data()
l_all.append(next(patient_dat_generator))

ses-right
RUNNIN SUBJECT_000_SESS_ses-right_SIGNAL_ECOG


In [54]:
l_all.append(next(patient_dat_generator))

In [55]:
len(l_all)

2

In [58]:
i = 0
while True:
    i +=1
    if (i % 60) == 0:
        print("break at i="+str(i))
        break
print("and back")

break at i=60
and back


In [12]:
PATH = "C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\MOVEMENT DATA\\ECoG_STN\\NN_Out\\006BestChpredictions_ECOG-ch-35-lat-IPS-ses-left.npy"

In [13]:
np.load(PATH, allow_pickle=True)

array({'y_pred_test': [array([0.09022155, 0.03381903, 0.0220023 , ..., 0.10388179, 0.10783441,
       0.10616177], dtype=float32), array([0.12789677, 0.12744671, 0.12647632, ..., 0.02634905, 0.1228833 ,
       0.12344892], dtype=float32), array([0.15403625, 0.15235761, 0.14889506, ..., 0.00866455, 0.00838803,
       0.00810571], dtype=float32)], 'y_test': [array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0.26174359, 0.55743707, 0.80283973, ..., 0.        , 0.        ,
       0.        ])], 'y_pred_train': [array([0.10394712, 0.10463816, 0.10486241, ..., 0.00895429, 0.00917452,
       0.00871366], dtype=float32), array([0.01110069, 0.0103773 , 0.00982634, ..., 0.01031327, 0.00904689,
       0.00845811], dtype=float32), array([0.02468543, 0.00946013, 0.00825727, ..., 0.04918476, 0.03546977,
       0.02947852], dtype=float32)], 'y_train': [array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])], 's

In [76]:
out_path_file = os.path.join(settings['out_path_process']+ \
                            settings['num_patients'][sub_idx]+'BestChpredictions_'+\
                            signal_+'-ch-'+str(ch_idx)+'-lat-'+str(laterality_)+'-'+str(subfolder[sess_idx])+'.npy')

In [77]:
out_path_file

'C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\\MOVEMENT DATA\\ECoG_STN\\NN_Out\\000BestChpredictions_ECOG-ch-0-lat-CON-right.npy'

In [79]:
if os.path.exists(PATH) is True:
    print("JA")

JA
